In [6]:
from io import StringIO
import pdfplumber
import jieba
import jieba.posseg as pseg #词性标注
import jieba.analyse as anls #关键词提取
from sklearn.cluster import KMeans
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
from docx import Document
from io import StringIO
import json
import re
import os
from collections import Counter
class Template():
    def __init__(self,type):
        self._type = type

    def re_match(self,texts):
        """匹配技术部分小节"""
        pattern = "第.*节"
        regex = re.compile(pattern)
        sections = []
        index = 0
        for text in texts:
            if len(regex.findall(text)) > 0:
                sections.append((index, text))
            #             print(text,index)
            index += 1
        return sections
    def get_sections(self,dirname):
        """获取小节"""
        sections = {}
        text_extract = {}
        for _, _, files in os.walk(dirname):
            for filename in files:
                if filename.split('.')[-1] == 'pdf':
                    texts = pdf_parse_text(dirname + '/' + filename)
                    text_extract[filename.split('.')[0]] = texts
                    sections_ = re_match(texts)
                    sections[filename.split('.')[0]] = sections_
                    print("{}小节获取完毕".format(filename))
        return sections, text_extract

def pdf_parse_table(path):
    """解析pdf文件中的表格"""
    origin = pdfplumber.open(path)
    tables = []
    for page in origin.pages:
        # 获取当前页面的全部表格
        table_page = [table for tables in page.extract_tables() for table in tables if len(table)!=0]
        tables.append(table_page)
    origin.close()
    return tables

def pdf_parse_text(path):
    """解析pdf为"""
    origin = pdfplumber.open(path)
    texts = []
    for page in origin.pages:
        text_page = [text.replace(' ','') for text in page.extract_text().split('\n') if len(text.replace(' ','')) != 0]
        texts.extend(text_page)
    origin.close()
    return texts

def re_match(texts):
    """匹配技术部分小节"""
    pattern = "第.*节"
    regex = re.compile(pattern)
    sections = []
    index = 0
    for text in texts:
        if len(regex.findall(text)) > 0:
            sections.append((index,text))
#             print(text,index)
        index+=1
    return sections

def get_sections(dirname):
    """获取小节"""
    sections = {}
    text_extract = {}
    for _,_,files in os.walk(dirname):
        for filename in files:
            if filename.split('.')[-1] == 'pdf':
                texts = pdf_parse_text(dirname+'/'+filename)
                text_extract[filename.split('.')[0]] = texts
                sections_ = re_match(texts)
                sections[filename.split('.')[0]] = sections_
                print("{}小节获取完毕".format(filename))
    return sections,text_extract

def get_exact_section(dirname,numberlist):
    """获取解析的小节列表"""
    sections = []
    for _,_,files in os.walk(dirname):
        for filename in files:
            if filename.split('.')[-1] == 'pdf':
                texts = pdf_parse_text(dirname+'/'+filename)
                try:
                    section_exact = texts[numberlist[filename.split('.')[0]][0]:numberlist[filename.split('.')[0]][1]]
                    sections.append((filename.split('.')[0],section_exact))
                except:
                    pass
                print("{}小节获取完毕".format(filename))
    return sections
def get_cross(sections):
    """获取公有的篇章"""
    set1 = {sectionname[3:] for _, sectionname in list(sections.values())[0]}
    for label, sections_ in sections.items():
        set2 = {sectionname[3:] for _, sectionname in list(sections_)}
        if (len(set2) > 0):
            # print(set2)
            set1 = set1 & set2
            print(set1)
    counter = Counter()
    for label, sections_ in sections.items():
        set2 = {sectionname[3:] for _, sectionname in list(sections_)}
        if (len(set2) > 0):
            set3 = set2 - set1
            counter.update(set3)
    summary = len(sections)
    for item, count in counter.items():
        if (count > summary * 0.5):
            set1.update({item})
    return set1
def get_section_matiral_index(sections,extracts:set):
    """
    获取章节的索引
    :param sections:
    :param extracts:
    :return:
    """
    result = {}
    for extract in extracts:
        section_matriels = []
        for label,sections_ in sections.items():
            regex = re.compile(extract)
            for index in range(len(sections_)):
                if len(regex.findall(sections_[index][1])):
                    start = sections_[index][0]
                    if index != len(sections_) - 1:
                        end = sections_[index + 1][0]
                    else:
                        end = start
                    if end - start > 1: # 过滤目录
                        section_matriels.append((label,(start,end)))
        result[extract] = section_matriels
    return result
def get_matiral(matriel_index,texts,extracts):
    """
    获得章节素材，进一步处理
    :param matriel_index:
    :param texts:
    :param extracts:
    :return:
    """
    matirals = {}
    for extract in extracts:
        temp = []
        for label,(start,end) in matriel_index[extract]:
            print(texts[label][start:end])
            temp.append(texts[label][start:end])
        matirals[extract] = temp
    return matirals

In [5]:
dirname='extract/'
sections,text_extract=get_sections(dirname)
#extracts=get_cross(sections)
set1 = {sectionname[3:] for _, sectionname in list(sections.values())[0]}
print(set1)
# matriel_index=get_section_matiral_index(sections,extracts)
# result=get_matiral(matriel_index,text_extract,extracts)

IndexError: list index out of range

In [36]:
result['供货范围、技术规格、参数与要求'][11]

IndexError: list index out of range

In [6]:
import re
pattern='采煤机'
regex=re.compile(pattern)
regex.findall('第2-2项：刮板运输机（SGZ1000/2*1000）')

[]

In [7]:
import json
import csv

"""
需求：将json中的数据转换成csv文件
"""
def csv_json(path):
    # 1.分别 读，创建文件
    json_fp = open(path, "r",encoding='utf-8')
    write_path=path.replace('json','csv')
    csv_fp = open(write_path, "w",encoding='utf-8-sig')

    # 2.提出表头和表的内容
    data_list = json.load(json_fp)

    sheet_title = data_list[0].keys()
     
    sheet_data = []
    for data in data_list:
        sheet_data.append(data.values())

    # 3.csv 写入器
    writer = csv.writer(csv_fp)

    # 4.写入表头
    writer.writerow(sheet_title)

    # 5.写入内容
    writer.writerows(sheet_data)

    # 6.关闭两个文件
    json_fp.close()
    csv_fp.close()


 

In [8]:
csv_json('../extract_dict_lists_采煤机.json')

In [2]:
import regex as re 
pattern_end_2='\W*第(一|二|三|四|五|六|七|八|九)节'
text='第二节 设别'
re.match(pattern_end_2,text)

<regex.Match object; span=(0, 3), match='第二节'>

In [3]:
stack=[]
if not stack:
    print('tet')

tet
